In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [3]:
chromedriver ='/Users/youngseolee/Downloads/chromedriver'
driver = webdriver.Chrome(chromedriver)

In [27]:
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='
url= base_url + quote('스마트폰')

In [5]:
driver.get(url)

In [12]:
# blog 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys
body = driver.find_element_by_tag_name('body')
for i in range(20):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [15]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [16]:
lis = soup.select('li.bx._svp_item')
len(lis)

62

In [18]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
date = li.select_one('.sub_time.sub_txt').get_text()

title, date, content

('[스마트폰] 삼성 갤럭시Z 플립3 언박싱 후기',
 '2021.08.16.',
 '진짜 역대급으로 나온 스마트폰, 삼성 갤럭시Z 플립3 언박싱 후기 함께 공개된 삼성 갤럭시Z 폴드3가 기대... 아무래도 기존 스마트폰과 거의 비슷한 비율과 함께 큰 화면을 반으로 접어서 가지고 다니면서 컴팩트하게...')

In [20]:
title_list, content_list, date_list = [],[],[]
for i,li in enumerate(lis):
    try:
        adv = li.select_one('a.link_ad').get_text()
        if adv.strip() == '광고':
            continue
    except:
        title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
        content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
        date = li.select_one('.sub_time.sub_txt').get_text()    

        title_list.append(title)
        content_list.append(content)
        date_list.append(date)
    

In [21]:
df = pd.DataFrame({
    'title':title_list, 'date': date_list, 'content':content_list
})
df.head()

,title,date,content
0,[스마트폰] 삼성 갤럭시Z 플립3 언박싱 후기,2021.08.16.,"진짜 역대급으로 나온 스마트폰, 삼성 갤럭시Z 플립3 언박싱 후기 함께 공개된 삼성..."
1,갤럭시 A03s의 출시소식과 상세 정보: 개발도상국용 초저가 스마트폰,7일 전,삼성이 인도시장 등 개발도상국용 초저가 스마트폰인 A0x 시리즈를 지난 2019년부...
2,에어팟 프로 안드로이드 스마트폰과 연결하는 방법,2021.08.21.,"그러면 이때 스마트폰 블루투스 메뉴로 가줍니다. 찾기를 눌러주면, 이렇게 검색이 되..."
3,갤럭시 굿락 루틴+ 스마트폰 배터리 충전 구간 알림,2021.08.08.,갤럭시 스마트폰을 사용하고 있다면 굿락 앱은 꼭 써볼 것을 추천드려요. 지속적인 업...
4,삼성 갤럭시노트10 가격 2021년에도 매력적인 스마트폰,2021.06.23.,"일상에서 스마트폰 사용량은 점점 늘어나고 있습니다. 카카오톡, 라인 같은 기본적인 ..."


In [24]:
df.shape

(60, 3)

In [23]:
df.to_csv('smartphone_blog.txt')

In [25]:
driver.close()